# AML Framework Code

In [1]:
# Import azureml sdk packages
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.estimator import Estimator
import azureml.core

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.19.0


In [2]:
# Set variables
cluster_name     = "larger-cluster"
experiment_name  = "fashion-mnist"
environment_file = "fashion-env.yml"
environment_name = "fashionenv"
training_script = 'fashion-mnist-code.py'

In [3]:
# Log into a workspace
try:
    ws = Workspace.from_config()
    print("Using workspace:",ws.name,"in region", ws.location)
except:
    print('Workspace not found')

Using workspace: cloudnativeml in region westeurope


In [4]:
# Choose computer cluster
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Using compute cluster:', cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2',
                                                           vm_priority='lowpriority',
                                                           min_nodes=0,
                                                           max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion(show_output=True)

Using compute cluster: larger-cluster


In [5]:
# Connect to the default datastore
ds = ws.get_default_datastore()

In [13]:
# Create an experiment or connect if it exists
exp = Experiment(workspace=ws, name=experiment_name)

In [9]:
# create an environment from conda yml file
fashionenv = Environment.from_conda_specification(name = environment_name ,
                                          file_path = environment_file )
print(fashionenv)

fashionenv.register(workspace=ws)

Environment(Name: fashionenv,
Version: None)


{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20200821.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "fashionenv",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "defaults"
            ],
            "depe

In [10]:
# List environments
envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("fashion"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

Name fashion-env
packages channels:
- anaconda
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-dataset-runtime[pandas,fuse]~=1.19.0
  - azureml-defaults~=1.19.0
- scikit-learn==0.22.1
- tensorflow==2.3.1
- keras
name: azureml_d57257a6f59123b47b31278c29479d2c

Name fashionenv
packages channels:
- defaults
dependencies:
- python=3.6.2
- tensorflow
- keras
- scikit-learn==0.22.1
- matplotlib
- pip
- pip:
  - azureml-sdk
name: azureml_1c440a5c6bb282f43b50436f72db0aa8



In [7]:
# Create training folder and move train.py file into it
import os
import shutil

script_folder = os.path.join(os.getcwd(), "fashion-mnist")
os.makedirs(script_folder, exist_ok=True)
shutil.copy(training_script, script_folder)

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/amyb-devmac/code/Users/amynic/fashion-mnist/code/fashion-mnist/fashion-mnist-code.py'

In [11]:
# create ScriptRunConfig object and connect environment
from azureml.core import ScriptRunConfig

args = ['--data-folder', '--regularization', 0.5]

src = ScriptRunConfig(source_directory=script_folder,
                      script=training_script, 
                      arguments=args,
                      compute_target=cluster,
                      environment=fashionenv)

In [14]:
# Submit a run
run = exp.submit(src)
aml_url = run.get_portal_url()
print(aml_url)

https://ml.azure.com/experiments/fashion-mnist/runs/fashion-mnist_1609761089_76043057?wsid=/subscriptions/a2a1fc9f-5671-4479-8922-ad16e34c0fdc/resourcegroups/cloud-native-ml/workspaces/cloudnativeml


In [15]:
# View Jupyter widget
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…